In [1]:
import tensorflow as tf
from cleverhans.tf2.attacks.fast_gradient_method import fast_gradient_method
from cleverhans.tf2.attacks.projected_gradient_descent import projected_gradient_descent
from cleverhans.tf2.attacks import carlini_wagner_l2
import matplotlib.pyplot as plt
import numpy as np
from keras.models import load_model
from PreprocessTool import getImage_wellClassified
from PreprocessTool import resnet50_deprocess
import PIL
from PreprocessTool import isAdversarial_ResNet20,isAdversarial_ResNet50
import os

In [2]:
#检查对抗样本的有效性并保存
def checkAEsAndSave(img_raws,img_advs,dataset,method,eps,norm,targeted):
    num=0
    dir="./CleverHansSample/"+method+"/"+dataset+"/"
    # 检查并创建新路径
    if not os.path.exists(os.path.dirname(dir)):
        os.makedirs(os.path.dirname(dir))
    for img_raw,img_adv in zip(img_raws,img_advs):
        if dataset=="Cifar10":
            flag,label_raw,label_adv=isAdversarial_ResNet20(img_raw,img_adv)
        elif dataset=="ImageNet":
            flag,label_raw,label_adv=isAdversarial_ResNet50(img_raw,img_adv)
        if flag:
            raw_path=dir+"raw"+"_"+str(num+1)+"_"+str(label_raw)+"_"+str(eps)+"_"+str(norm)+"_"+str(targeted)+".jpeg"
            adv_path=dir+"adv"+"_"+str(num+1)+"_"+str(label_adv)+"_"+str(eps)+"_"+str(norm)+"_"+str(targeted)+".jpeg"
            img_raw.save(raw_path)
            img_adv.save(adv_path)
            num+=1     
    print("生成对抗样本数量：",num)         
    print("生成对抗样本成功率为：",num/len(img_raws))

In [3]:
def CleverHansAttack(dataset,
                    picNum,
                    method,
                    eps=0.5,
                    norm=2,
                    eps_iter=None,
                    nb_iter=None,
                    confidence=0.5,
                    learning_rate=0.01,
                    binary_search_steps=10,
                    max_iterations=1000, 
                    abort_early=True,
                    ):
    if dataset=="ImageNet":
        #获取模型分类正确的图像数据
        labelName,imgs,imgData,predictions,count=getImage_wellClassified(picNum,"ImageNet")
        model=tf.keras.applications.resnet50.ResNet50(weights='imagenet')
        max_index = np.argmax(predictions, axis=1)
        #对抗样本的最大最小值
        clip_min =np.array([-103.939, -116.779, -123.68],dtype="float32")
        clip_max =np.array( [255 - 103.939, 255 - 116.779, 255 - 123.68],dtype="float32")
    elif dataset=="Cifar10":
        #获取模型分类正确的图像数据
        labelName,imgs,imgData,predictions,count=getImage_wellClassified(picNum,"Cifar10")
        model=load_model("D:/Adversarial Attack/testModel/cifar10_ResNet20.hdf5")
        #对抗样本的最大最小值
        clip_min =0
        clip_max =1
    #获得真实标签
    max_index = np.argmax(predictions, axis=1)
    ground_truth = np.zeros_like(predictions)
    ground_truth[np.arange(len(predictions)), max_index] = 1
    if method!="C&W":
        if method=="fast_gradient_method":
            img_advs=fast_gradient_method(model_fn=model,
                                    x=tf.stack(imgData),
                                    eps=eps,norm=norm,
                                    clip_min=clip_min,clip_max=clip_max,
                                    )
            
        
        elif method=="projected_gradient_descent":
            img_advs=projected_gradient_descent(model_fn=model,
                                        x=tf.stack(imgData),
                                        eps=eps,
                                        eps_iter=eps_iter,
                                        nb_iter=nb_iter,
                                        norm=norm)
        #预处理逆变换，并转为Image
        img_advs_deprocessed=[]
        for i in img_advs:
            if dataset=="ImageNet":
               img_advs_deprocessed.append(PIL.Image.fromarray(resnet50_deprocess(i).numpy().astype('uint8')))
            elif dataset=="Cifar10":
               img_advs_deprocessed.append(PIL.Image.fromarray((i*255).numpy().astype('uint8')))
        #检查并保存
        checkAEsAndSave(img_raws=imgs,
                        img_advs=img_advs_deprocessed,
                        dataset=dataset,
                        method=method, 
                        eps=eps,
                        norm=norm,
                        targeted=False)
    else:
        #保存cw攻击的结果
        num=0
        dir="./CleverHansSample/"+method+"/"+dataset+"/"
        # 检查并创建新路径
        if not os.path.exists(os.path.dirname(dir)):
            os.makedirs(os.path.dirname(dir))
        for i,imgdata in enumerate(imgData):
            img_raw=np.clip(imgdata, a_min=clip_min, a_max=clip_max)
            cw = carlini_wagner_l2.CarliniWagnerL2(model_fn=model, 
                                                targeted=False,
                                                confidence=confidence, 
                                                learning_rate=learning_rate,
                                                binary_search_steps=binary_search_steps,
                                                max_iterations=max_iterations,
                                                abort_early=abort_early,
                                                clip_min=clip_min,
                                                clip_max=clip_max)
            img_adv=cw.attack(np.expand_dims(img_raw, axis=0))
            img_adv=np.squeeze(img_adv)
            if dataset=="ImageNet":
               img_adv_deprocessed=PIL.Image.fromarray(resnet50_deprocess(img_adv).astype('uint8'))
               flag,label_raw,label_adv=isAdversarial_ResNet50(imgs[i],img_adv_deprocessed)
            elif dataset=="Cifar10":
               img_adv_deprocessed=PIL.Image.fromarray((img_adv*255).astype('uint8'))
               flag,label_raw,label_adv=isAdversarial_ResNet20(imgs[i],img_adv_deprocessed)
            if flag:
                raw_path=dir+"raw"+"_"+str(num+1)+"_"+str(label_raw)+"_"+str(confidence)+"_"+str(norm)+"_"+str("False")+".jpeg"
                adv_path=dir+"adv"+"_"+str(num+1)+"_"+str(label_adv)+"_"+str(confidence)+"_"+str(norm)+"_"+str("False")+".jpeg"
                imgs[i].save(raw_path)
                img_adv_deprocessed.save(adv_path)
                num+=1
               

            
    
    
        

    



In [4]:
# CleverHansAttack(dataset="ImageNet",picNum=50,method="fast_gradient_method",eps=0.1,norm=2)
# CleverHansAttack(dataset="Cifar10",picNum=50,method="fast_gradient_method",eps=0.1,norm=2)

In [5]:
# CleverHansAttack(dataset="ImageNet",picNum=50,method="projected_gradient_descent",eps=0.1,norm=2,eps_iter=0.02,nb_iter=100)
# CleverHansAttack(dataset="Cifar10",picNum=50,method="projected_gradient_descent",eps=0.1,norm=2,eps_iter=0.02,nb_iter=100)

In [6]:
CleverHansAttack(dataset="ImageNet",picNum=50,method="C&W",confidence=0.5,learning_rate=0.01,binary_search_steps=15,max_iterations=1000,abort_early=True)
CleverHansAttack(dataset="Cifar10",picNum=50,method="C&W",confidence=0.5,learning_rate=0.01,binary_search_steps=15,max_iterations=1000,abort_early=True)

val_id:4254, ILSVRC_ID:614, WIND:n01978455
val_id:30077, ILSVRC_ID:583, WIND:n03075370
val_id:48048, ILSVRC_ID:937, WIND:n03866082
val_id:1667, ILSVRC_ID:494, WIND:n01629819
val_id:30741, ILSVRC_ID:360, WIND:n09468604
val_id:13831, ILSVRC_ID:223, WIND:n02672831
val_id:34367, ILSVRC_ID:725, WIND:n03459775
val_id:5894, ILSVRC_ID:629, WIND:n02190166
val_id:45013, ILSVRC_ID:596, WIND:n03933933
val_id:20188, ILSVRC_ID:922, WIND:n03733281
val_id:40179, ILSVRC_ID:931, WIND:n03935335
val_id:7959, ILSVRC_ID:504, WIND:n03876231
val_id:32252, ILSVRC_ID:856, WIND:n03141823
val_id:8494, ILSVRC_ID:940, WIND:n02865351
val_id:8753, ILSVRC_ID:368, WIND:n09288635
val_id:1985, ILSVRC_ID:615, WIND:n01980166
val_id:8899, ILSVRC_ID:672, WIND:n04596742
val_id:10171, ILSVRC_ID:464, WIND:n01677366
val_id:20853, ILSVRC_ID:285, WIND:n03977966
val_id:19845, ILSVRC_ID:785, WIND:n02807133
val_id:2520, ILSVRC_ID:250, WIND:n04389033
val_id:19359, ILSVRC_ID:918, WIND:n03630383
val_id:27572, ILSVRC_ID:595, WIND:n038146